In [1]:
import pandas as pd

In [2]:
benchmark1017 = r'C:\Users\jiayue.tan\OneDrive - YTL\Workstation\dot-helper\input\benchmark\Request_for_specific_language_single.json'
df_benchmark1017 = pd.read_json(benchmark1017)
df_benchmark1017.head(1)

,input,target_language,metadata
0,"[{'role': 'user', 'content': 'Senang je ni, ex...",ms,"{'turn': 'single', 'input_language': 'Casual M..."


In [3]:
import re

def get_total_word_count(question: str, answer: str, reason: str = "") -> int:
    combined_text = f"{question}{answer}{reason}"
    
    english_number_words = re.findall(r"[a-zA-Z0-9']+", combined_text)
    english_number_word_count = len(english_number_words)
    
    chinese_chars = re.findall(r'[\u4e00-\u9fff]', combined_text)
    chinese_char_count = len(chinese_chars)
    
    total_count = english_number_word_count + chinese_char_count
    return total_count

def get_turn_languages_standard(data_list):
    languages = []
    for item in data_list[1:]:
        metadata = item.get('metadata')
        if metadata:
            languages.append(metadata.get('language', 'N/A'))
    return languages

def flag_if_list_contains_chinese(language_list: list) -> int:
    
    if not isinstance(language_list, list):
        return 0
    for item in language_list:
        if isinstance(item, str) and "chinese" in item.lower():
            return 1
    return 0

df_benchmark1017['original_id'] = "benchmark_langfol1017_" + (df_benchmark1017.index + 1).astype(str).str.zfill(5)
df_benchmark1017['sft_round'] = "Benchmark-LanguageFollowing"
df_benchmark1017['question'] = df_benchmark1017['prompt']
df_benchmark1017['answer'] = df_benchmark1017['response']

df_benchmark1017['language'] = df_benchmark1017['reference'].apply(lambda x:x.get('language'))
# The original lambda, modified to start from the second item

# 1. Define a function that does the work for a single row's reference dictionary
def extract_languages_from_reference(ref_dict):
    """
    Safely extracts 'turn_language' from a single reference dictionary.
    Returns a list of languages or ['N/A'] if no data is found.
    """
    # Get the list of metadata, defaulting to an empty list if the key is missing
    metadata_list = ref_dict.get('all_turn_metadata', [])
    
    # If the list is empty, return a list with a single 'N/A' placeholder
    if not metadata_list:
        return ['N/A']
        
    # If the list is not empty, use the lambda to extract languages
    return [item.get('metadata', {}).get('turn_language', 'N/A') for item in metadata_list]

df_benchmark1017['turn_languages'] = df_benchmark1017['reference'].apply(extract_languages_from_reference)
df_benchmark1017['chinese_flag'] = df_benchmark1017['turn_languages'].apply(flag_if_list_contains_chinese)

df_benchmark1017['style'] = df_benchmark1017['reference'].apply(lambda x: x.get('style'))
df_benchmark1017['rendered_history'] = df_benchmark1017['history'].apply(render_messages_safe)

df_benchmark1017['reason'] = df_benchmark1017['rendered_history']
df_benchmark1017['task'] = "LANGUAGE: " + df_benchmark1017['language'].astype(str) + " | STYLE: " + df_benchmark1017['style'].astype(str) + " | SOURCE: " + df_benchmark1017['source']
df_benchmark1017['domain'] = df_benchmark1017['language']
df_benchmark1017['total_word_count'] = df_benchmark1017.apply(lambda row: get_total_word_count(row['question'], row['answer'], row['reason']), axis=1)

df_benchmark1017 = df_benchmark1017[['original_id','sft_round','question','answer','reason','turn_languages','chinese_flag','task','domain','style','total_word_count']]

def classify_wordcount_class(total_words_count):
    if total_words_count <= 600:
        return 'short(1-600)'
    elif total_words_count>=601 and total_words_count <= 1100:
        return 'medium(601-1100)'
    elif total_words_count>=1101 and total_words_count <= 1900:
        return 'long(1101-1900)'
    elif total_words_count>=1901:
        return 'very_long(>1901)'
    else: 
        return 'you_tell_me'
    
df_benchmark1017['wordcount_class'] = df_benchmark1017['total_word_count'].apply(classify_wordcount_class)
df_benchmark1017.groupby(['wordcount_class']).size().reset_index(name='counts').sort_values(by='counts', ascending=False)

df_benchmark1017.head(2)

KeyError: 'prompt'